In [8]:
"""
Exercise 01: Dynamic Prompts - Starter Code
===========================================
Build an agent with context-aware system prompts.

LEARNING GOALS:
- Import and use @dynamic_prompt decorator
- Read from ModelRequest to access state
- Build adaptive prompts based on context
"""

import os
from datetime import datetime
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

load_dotenv()

model = init_chat_model("gpt-4o-mini", model_provider="openai")


# =============================================================================
# TODO 1: Import Dynamic Prompt Components
# =============================================================================
# Import from langchain.agents.middleware:
# - dynamic_prompt (decorator)
# - ModelRequest (type for the request object)
# =============================================================================

from langchain.agents.middleware import dynamic_prompt, ModelRequest

In [9]:

# =============================================================================
# TODO 2: Create Tools
# =============================================================================

@tool
def explain_concept(topic: str) -> str:
    """Explain a programming concept."""
    response = model.invoke(f"Explain the programming concept: {topic}. Be clear and concise.")
    return response.content


@tool
def write_code(description: str) -> str:
    """Write code for a given task description."""
    response = model.invoke(f"Write code for: {description}. Provide complete, working code.")
    return response.content

In [10]:

# =============================================================================
# TODO 3: Implement Dynamic Prompt Middleware
# =============================================================================
# Create a function decorated with @dynamic_prompt that:
# 1. Receives a ModelRequest parameter
# 2. Reads experience_level from request.state (default: "intermediate")
# 3. Gets time of day from datetime.now().hour
# 4. Checks conversation length via len(request.messages)
# 5. Returns a customized system prompt string
#
# Adaptations to implement:
# - Beginner: simple language, analogies, step-by-step
# - Intermediate: balanced explanations with code
# - Expert: concise, technical terms freely
# - Morning: cheerful greeting
# - Long conversations: more concise responses
#
# EXPERIMENT: Add language preference adaptation
# EXPERIMENT: Add domain-specific adaptations (web dev vs data science)
# =============================================================================

@dynamic_prompt
def adaptive_prompt(request: ModelRequest) -> str:
    experience = request.state.get("experience_level", "intermediate")
    hour = datetime.now().hour
    message_count = len(request.messages)
    
    greeting = ""
    if 5 <= hour < 12:
        greeting = "Good morning! "
    
    experience_style = ""
    if experience == "beginner":
        experience_style = "Use simple language, analogies, and step-by-step explanations. Avoid jargon unless you explain it first."
    elif experience == "expert":
        experience_style = "Be concise and use technical terms freely. Assume deep knowledge."
    else:
        experience_style = "Provide balanced explanations with code examples. Use technical terms but explain when helpful."
    
    conciseness = ""
    if message_count > 10:
        conciseness = " Keep responses concise as this is a longer conversation."
    
    prompt = f"{greeting}You are a helpful programming assistant. {experience_style}{conciseness}"
    return prompt

In [11]:

# =============================================================================
# TODO 4: Create the Agent
# =============================================================================
# Create an agent that:
# - Uses your tools
# - Uses your adaptive_prompt middleware
# - Does NOT include checkpointer (for Studio export)
# =============================================================================

agent = create_agent(
    model=model,
    tools=[explain_concept, write_code],
    middleware=[adaptive_prompt],
    name="adaptive_programming_agent"
)

In [12]:

# =============================================================================
# CLI Testing
# =============================================================================
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

cli_agent = create_agent(
    model=model,
    tools=[explain_concept, write_code],
    middleware=[adaptive_prompt],
    checkpointer=checkpointer,
    name="adaptive_programming_agent_cli"
)

def test_agent(experience_level: str, question: str):
    config = {
        "configurable": {
            "thread_id": f"test_{experience_level}",
            "state": {"experience_level": experience_level}
        }
    }
    
    result = cli_agent.invoke(
        {"messages": [{"role": "user", "content": question}]},
        config
    )
    
    return result["messages"][-1].content

print("Testing with different experience levels:")
print("=" * 60)

question = "What is a decorator in Python?"

print(f"\nQuestion: {question}\n")
print("-" * 60)

print("\nBEGINNER response:")
print("-" * 60)
beginner_response = test_agent("beginner", question)
print(beginner_response)

print("\n\nEXPERT response:")
print("-" * 60)
expert_response = test_agent("expert", question)
print(expert_response)

print("\n\nINTERMEDIATE response:")
print("-" * 60)
intermediate_response = test_agent("intermediate", question)
print(intermediate_response)

Testing with different experience levels:

Question: What is a decorator in Python?

------------------------------------------------------------

BEGINNER response:
------------------------------------------------------------
In Python, a **decorator** is a design pattern that modifies or enhances functions or methods without altering their actual code. They allow you to add functionality to existing functions in a clean and reusable way.

### Key Concepts:

1. **First-Class Functions**: In Python, functions are first-class citizens, meaning they can be passed as arguments, returned from other functions, and assigned to variables. This characteristic allows decorators to take a function as input.

2. **Syntax**: Decorators use the `@decorator_name` syntax to apply them to a function.

3. **Function Wrapping**: A typical decorator function defines an inner function (often called a wrapper) that adds functionality before or after the original function call.

### Example:

Here's a simpl